# Naive Bayes On The Iris Dataset

In [ ]:
from csv import reader
from math import sqrt
from math import exp
from math import pi
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_recall_curve, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

## Split the dataset by class values, returns a dictionary

In [ ]:
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if (class_value not in separated):
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated

## Calculate the mean of a list of numbers

In [ ]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

## Calculate the standard deviation of a list of numbers

In [ ]:
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return sqrt(variance)

## Calculate the mean, stdev and count for each column in a dataset

In [ ]:
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del(summaries[-1])
    return summaries

## Split dataset by class then calculate statistics for each row

In [ ]:
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries

## Calculate the Gaussian probability distribution function for x

In [ ]:
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

## Calculate the probabilities of predicting each class for a given row

In [ ]:
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

## Predict the class for a given row

In [ ]:
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

# Make a prediction with Naive Bayes Classifier Model on Iris Dataset

## load dataset

In [ ]:
filename = '../../images/prob/iris.csv'   
dataset = pd.read_csv(filename)

In [ ]:
dataset.head()

In [ ]:
dataset.variety.value_counts()

## preprocess dataset

In [ ]:
label_map = {"Virginica":0, "Setosa":1, "Versicolor":2}

In [ ]:
dataset.variety = dataset.variety.apply(lambda x: label_map[x])

In [ ]:
dataset.head()

In [ ]:
data = dataset.as_matrix()

In [ ]:
data.shape

## fit model on full data (we aren't splitting data in train/val/test parts here)

In [ ]:
model = summarize_by_class(data)

## Test Model on full data

In [ ]:
predictions = [predict(model, data[row_idx, :4]) for row_idx in range(data.shape[0])]

In [ ]:
targets = data[:, 4]

### Get Scores

In [ ]:
f1 = f1_score(y_true=targets, y_pred=predictions, average='macro')
precision = precision_score(y_true=targets, y_pred=predictions, average='macro')
recall = recall_score(y_true=targets, y_pred=predictions, average='macro')

In [ ]:
print(f"Scores:\nf1: {f1}\nprecision:{precision}\nrecall:{recall}")

### Confusion Matrix

In [ ]:
label_keys = ["Virginica", "Setosa", "Versicolor"]

In [ ]:
label_values = [0,1,2]

In [ ]:
conf_matrix = confusion_matrix(y_true=targets, y_pred=predictions, labels=label_values)

In [ ]:
conf_matrix_df = pd.DataFrame(conf_matrix, columns=label_keys, index=label_keys)

In [ ]:
conf_matrix_df

In [ ]:
f, ax = plt.subplots(figsize=(11, 15)) 
heatmap = sns.heatmap(conf_matrix_df, 
                      square = True,
                      linewidths = .5,
                      cmap = 'coolwarm',
                      cbar_kws = {'shrink': .4, 
                                'ticks' : [-1, -.5, 0, 0.5, 1]},
                      vmin = -1, 
                      vmax = 1,
                      annot = True,
                      annot_kws = {"size": 12})

#add the column names as labels
ax.set_yticklabels(conf_matrix_df.columns, rotation = 0)
ax.set_xticklabels(conf_matrix_df.columns)
sns.set_style({'xtick.bottom': True}, {'ytick.left': True})
ax.set(ylabel='True label',
           xlabel='Predicted label');

## define a new record

In [ ]:
row = [5.7,2.9,4.2,1.3]

## predict the label

In [ ]:
label = predict(model, row)
print('Data=%s, Predicted: %s' % (row, label))

# More Resources

[all about naive bayes](https://towardsdatascience.com/all-about-naive-bayes-8e13cef044cf)

[naive bayes classifier (wiki)](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)

[naive bayes explained (towardsdatascience)](https://towardsdatascience.com/naive-bayes-explained-9d2b96f4a9c0)